<a href="https://colab.research.google.com/github/Theskrtnerd/gen-ai-photoshoots/blob/main/gen_ai_photoshoots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#@title Installing all the dependencies for the project
%%capture
%cd /content/
# Clone the repository if it doesn't exist
import os
if not os.path.isdir('app'):
    !git clone https://github.com/Theskrtnerd/gen-ai-photoshoots app
else:
    print("Directory 'app/' already exists. Skipping clone.")
!sudo apt install pipx
!pipx ensurepath
!cp -r /root/.local/bin /usr/local
%cd app
!sudo apt-get install python3.12
!pipx install --python python3.12 poetry
!poetry install

In [ ]:
#@title Installing all the dependencies for the project
# @markdown Please go to [Ngrok Dashboard](https://dashboard.ngrok.com/get-started/your-authtoken), sign up (if you've not already) and get your auth token.

# @markdown Enter the auth token here:
ngrok_key = "2gssWVZmNGCsLcs8n3NAulvIqYE_5DupK4SWUHnGbWjAe7WAw" # @param {type:"string"}
%cd /content/app
!pip install pyngrok > /dev/null
from pyngrok import conf, ngrok
conf.get_default().region = "us"
conf.get_default().auth_token = ngrok_key
port = 8501
public_url = ngrok.connect(port).public_url
print("Here's your public URL: ", public_url)
!poetry run streamlit run gen_ai_photoshoots/main.py > /dev/null &

/content/app
Here's your public URL:  https://9a84-34-142-212-198.ngrok-free.app
